<h1>Determine the intersection of a probability density function graph</h1>

In [ ]:
#determine min and max for both graphs
#min,max,bins defined
#need to filter and subset each dataframe to get the min and max
min_5 = df[(df['date']==date) & (df['Class'] == 5)][kpi].min()
max_5 = df[(df['year_month']==date) & (df['Class'] == 5)][kpi].max()

min_1 = df[(df['date']==date) & (df['Class'] == 1)][kpi].min()
max_1 = df[(df['date']==date) & (df['Class'] == 1)][kpi].max()

bins=1000
# pull lower of 2 maximums and higher of 2 minimums
# goal is to segment the graphs to be the same size (x,y), so that we can subract 1 graph from another and grab the intersection at 0.
# by setting the min and max, can set up a new x-axis for the time series to have the same shape and same start and end point, allowing us to subtract
mina=pd.Series([min_1, min_5]).max()
maxa=pd.Series([max_1, max_5]).min()


#plot density plots for both Class=5 and Class=1, allowing us to generate density plots
#from the plots generated, use get_lines() and get_data() to get the x and y values used to create plot
#plot both Class=1 and Class=5
# use get_data() to grab the x,y coordinates
ax = df[(df['date']==date) & (df['Class'] == 5)][kpi].plot.density()
kde_lines = ax.get_lines()[-1]
kde_x, kde_y = kde_lines.get_data()

ax2 = df[(df['date']==date) & (df['Class'] == 1)][kpi].plot.density()
kde_lines2 = ax2.get_lines()[-1]
kde_x2, kde_y2 = kde_lines2.get_data()

#code below removes plots generated by using plot.density function
#otherwise, plots will still generate and be shown below
plt.clf()

#given the x and y coordinates, use interpolae function to approximate the function
xy1 = interp.interp1d(kde_x,kde_y)
xy2 = interp.interp1d(kde_x2,kde_y2)

#if step function matches maxa, then will run into issues
#multiply maxa by .001 to ensure maxa!= step function
xnew = np.arange(mina,maxa,maxa*.001)

#given the interpolate and range(np.range), fit curve within dimensions, so we can subtract at the right spot to find intersection
y1new = xy1(xnew)
y2new = xy2(xnew)

#determine where the intersect(s) are 
#np.sign(): pulls the sign(- or positive)
#np.diff(): compute the difference and find where there is a difference (positive)
#np.argwhere(): find where the intersection is
#flatten(): collapse into 1 dimension
zero_crossing_index = np.argwhere(np.diff(np.sign(y2new - y1new))).flatten()

#print all intersections (if any)
#print(xnew[zero_crossing_index])

#grab intersection point
thr = xnew[zero_crossing_index[0]]

if score_of_interest==5:
    main_score=5
    second_score=1

if score_of_interest==1:
    main_score=1
    second_score=5

#useable plot
#given the intercept, examine the distribution (Class=1 or Class=5) and determine which side to interpret
#plot both Class=1 and Class=5
# use get_data() to grab the x,y coordinates
ax = df[(df['year_month']==date) & (df['Class'] == main_score)][kpi].plot.density()
kde_lines = ax.get_lines()[-1]
kde_x, kde_y = kde_lines.get_data()

ax2 = df[(df['year_month']==date) & (df['Class'] == second_score)][kpi].plot.density()
kde_lines2 = ax2.get_lines()[-1]
kde_x2, kde_y2 = kde_lines2.get_data()

#have to embed after graphs are created above
if score_of_interest==5:
    plt.legend(["Score=5","Score=1"])

if score_of_interest==1:
    plt.legend(["Score=1","Score=5"])



#plot threshold
plt.axvline(x=thr, color='b', linestyle='--')

# Use Numpy mask to filter the lines for region 
if direction == 'left':
    mask = kde_x < thr
    filled_x, filled_y = kde_x[mask], kde_y[mask]
    dir = 'less'

if direction == 'right':
    mask = kde_x > thr
    filled_x, filled_y = kde_x[mask], kde_y[mask]
    dir = 'greater'

# Shade the partial region 
ax.fill_between(filled_x, y1=filled_y)


plt.title(kpi + " Density Distributions for " + date, pad=20, loc='center', fontsize=18)
plt.xlabel(kpi, fontsize=16)
plt.ylabel("Probability", labelpad=20, fontsize=16)
plt.show()

# Area of the blue region
#area can be used for statistical statement
area = np.trapz(filled_y, filled_x)